In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium

*Load Data*

In [ ]:
file_path = "flight_data.csv"
df = pd.read_csv(file_path)
print("Data shape: ", df.shape)
df.head()

*Inspect data*

In [ ]:
print(df.columns)
print(df.info())
print(df.describe())

*Data Cleaning*

In [8]:
df = df.drop("sensors", axis=1)
df = df.drop_duplicates()
print(df.shape)

(11478, 18)


*Convert UNIX time to datetime*

In [ ]:
for col in ["time_position","last_contact"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], unit='s', errors="coerce")
print(df.head())

*Handle missing values*

In [ ]:
print(df.isna().sum())
df = df.dropna(subset=["longitude", "latitude"])

**EXPLORATORY ANALYSIS**

*Flights per country*